Based off on longinvest's post[1] on bogleheads.

[1]: https://www.bogleheads.org/forum/viewtopic.php?f=10&t=179425

A ladder is built from 10-year maturity down as short as you want.

We need the current interest rate to determine the yield on the shortest maturity we hold :(

longinvest uses data from Shiller up to 1954 and then uses FRED 1 year Treasury data after that.

We need monthly data....

* FRED has 1-month Treasury data at https://fred.stlouisfed.org/series/GS1M but it only goes back to 2001
* FRED has 3-month at https://fred.stlouisfed.org/series/DGS3MO going back to 1982
* FRED has 1-year (monthly data) going back to 1953 at https://fred.stlouisfed.org/series/GS1

longinvest builds a 10-2 ladder using 9 bonds: 10-year maturity, 9-year maturity, etc down to 2-year maturity

Except there's a bootstrap period in the first decade...

So let's try building a similar ladder but with monthly granularity. That means instead of 9 bonds we want (12 * 9) + 11 = 119 bonds. Every month we sell a bond that is about to mature (that has 1 month left) and buy a brand new 10 year bond at the current rate.

Let's start in July 2001 so we can use the FRED monthly data.

The starting FRED rate is 3.67
Shiller's 10-year yield is 5.24


In [132]:
import numpy
from collections import deque
import pandas

In [161]:
#STEPS = 9 * 12 + 11
STEPS = 9

def interpolate_rates(shortest, longest, steps=STEPS):
    rates = []

    # this is just a linear interpolation between the two
    for cur_step in range(1, steps+1):
        rate = shortest + ((longest - shortest) * (cur_step - 1) / steps)
        rates.append(rate)
    # we don't want to approximate this...use the exact number
    rates.append(longest)
    return rates

# This sucks...if I want to calculate annual (to check vs longinvest)
# I need to change this...
def a2m(annual):
    """ Convert an annual rate to a monthly rate"""
    #return pow(1 + annual, 1/12) - 1
    return annual

class Bond:
    def __init__(self, face_value, yield_pct, maturity, sell_at_term=1):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        self.sell_at_term = sell_at_term
        
    def __repr__(self):
        return ('M: %d | Y: %.2f%% | FV: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))
        
    def gen_payment(self, current_rate):
        # current_rate is the shortest rate currently available.
        # it is an annual rate so we need to convert it to a monthly rate
        
        #self.maturity -= 1
        my_yield = self.face_value * a2m(self.yield_pct)
        
        # when we are almost mature, sell the entire bond.
        # We aren't quite mature so we can't sell at face value,
        # instead we need to calculate our present value and sell for that
        if self.maturity <= self.sell_at_term:
            pv = -numpy.pv(a2m(current_rate), 1, my_yield, self.face_value)
            my_yield += pv
        return my_yield
    
    def value(self, rates):
        value = numpy.pv(a2m(rates[self.maturity - 1]), self.maturity, self.face_value * a2m(self.yield_pct), self.face_value)
        return -value
    
def get_nav(ladder, rates):
    return sum((b.value(rates) for b in ladder))

def get_payments(ladder, short_rate):
    return sum((b.gen_payment(short_rate) for b in ladder))

def advance(ladder, short_rate, long_rate, steps=STEPS):
    payments = get_payments(ladder, short_rate)

    sold_bond = ladder.popleft()
    new_bond = Bond(payments, long_rate, steps+1)
    ladder.append(new_bond)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest does...)
    yield_curve = interpolate_rates(short_rate, long_rate)
    nav = get_nav(ladder, yield_curve)

    return (ladder, payments, nav)

def reduce_maturity(ladder):
    for b in ladder:
        b.maturity -= 1
    return ladder

def bootstrap(short_yield, long_yield, steps=STEPS):
    ladder = deque()
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)
    for i in range(steps):
        face_value = pow(1 + long_yield, i) * starting_face_value
        b = Bond(face_value, long_yield, i+1)
        ladder.append(b)
    return ladder

def loop(bootstrap_rates, rates):
    ladder = bootstrap(bootstrap_rates[0], bootstrap_rates[1], steps=9)
    for (_, (short, long)) in rates:
        (ladder, payments, nav) = advance(ladder, short, long)
        #print("%.2f" % (nav,))
        reduce_maturity(ladder)

In [162]:
df = pandas.read_csv('longinvest_bond_rates.csv')

In [163]:
loop(df.iloc[0], df.iterrows())

It matches the 10-2 spreadsheet perfectly up until 1954...at which point longinvest starts using FRED data where available (e.g. for 1-year, 3-year, and 5-year) instead of continuing with linear interpolation.

TODO

- First, generalise things. Make it so I can do 10-2, 10-4, 4-2, 3-2, 30-11, 20-2.
- Handle the 2-year fund. What's this?
- Handle "Bond Ladder". The "Bond Fund" sells prior to maturity. The "Bond Ladder" holds to maturity.
- Handle FRED data, when it is available.
- Once everything works for annual data, try monthly.